In [12]:
###### Giada Barzaghi - August 2021
### Modeling aviation networks: hybrid networks 
### Goal 1. integrating the concept of itinerary 
### Goal 2. insert some more realistic values for the variables in the model (eg. form Seymour, EMEP/EEA spreadsheet)
### Goal 3: REPRESENT YOUR NETWORK GRAPHICALLY + other graphs 

# SETTING UP WORKING ENVIRONMENT
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyomo.environ import *

In [13]:
# CREATING SETS AND ASSIGNING VALUES TO VARIABLES

# set of airports 
N = [n for n in range(1,10)]

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [14]:
# set of markets ---> note: remember difference city-pair / airport-pair
M = [(n1, n2) 
    for n1 in N 
    for n2 in N 
     if n1!=n2]
#print(M)

In [15]:
# set of flight-legs
F = M
#print(F)

In [16]:
# set of itineraries (Seba's method) ---> DISREGARD FOR NOW
I1 = [(n1, n2, n3)
    for n1 in N
    for n2 in N
    for n3 in N
    if n1!=n2
    if n1!=n3
    if n3!=n2]
#print(I1)

In [17]:
# here the last airport can be the same as the staring one ----> DISREGARD FOR NOW
I2 = []
for f in F:
    for m in M:
        if f[-1]== m[0]:
            c = (m[-1],)
            b = f+c
            I2.append(b)
        
#print(I2)

In [18]:
# One - stop itineraries
I3 = []
for f in F:
    for m in M:
        if f[-1]== m[0] and f[0]!=m[-1]:
            c = (m[-1],)
            b = f+c
            I3.append(b)
        
#print(I3)

In [88]:
# 2 - stop itineraries
I4 =[]
for i in I3:
    for m in M:
        if m[-1]==i[0] and m[0] not in i:
            c = list(i)
            c.pop(0)
            c = tuple(c)
            b = m+c
            I4.append(b)
#print(I4)

[(4, 1, 2, 3), (5, 1, 2, 3), (6, 1, 2, 3), (7, 1, 2, 3), (8, 1, 2, 3), (9, 1, 2, 3), (3, 1, 2, 4), (5, 1, 2, 4), (6, 1, 2, 4), (7, 1, 2, 4), (8, 1, 2, 4), (9, 1, 2, 4), (3, 1, 2, 5), (4, 1, 2, 5), (6, 1, 2, 5), (7, 1, 2, 5), (8, 1, 2, 5), (9, 1, 2, 5), (3, 1, 2, 6), (4, 1, 2, 6), (5, 1, 2, 6), (7, 1, 2, 6), (8, 1, 2, 6), (9, 1, 2, 6), (3, 1, 2, 7), (4, 1, 2, 7), (5, 1, 2, 7), (6, 1, 2, 7), (8, 1, 2, 7), (9, 1, 2, 7), (3, 1, 2, 8), (4, 1, 2, 8), (5, 1, 2, 8), (6, 1, 2, 8), (7, 1, 2, 8), (9, 1, 2, 8), (3, 1, 2, 9), (4, 1, 2, 9), (5, 1, 2, 9), (6, 1, 2, 9), (7, 1, 2, 9), (8, 1, 2, 9), (4, 1, 3, 2), (5, 1, 3, 2), (6, 1, 3, 2), (7, 1, 3, 2), (8, 1, 3, 2), (9, 1, 3, 2), (2, 1, 3, 4), (5, 1, 3, 4), (6, 1, 3, 4), (7, 1, 3, 4), (8, 1, 3, 4), (9, 1, 3, 4), (2, 1, 3, 5), (4, 1, 3, 5), (6, 1, 3, 5), (7, 1, 3, 5), (8, 1, 3, 5), (9, 1, 3, 5), (2, 1, 3, 6), (4, 1, 3, 6), (5, 1, 3, 6), (7, 1, 3, 6), (8, 1, 3, 6), (9, 1, 3, 6), (2, 1, 3, 7), (4, 1, 3, 7), (5, 1, 3, 7), (6, 1, 3, 7), (8, 1, 3, 7), (9, 1

In [103]:
# Merging all itineraries (with 0, 1 or 2 stops) together to get the complete set of itineraries 
I = F+I3+I4
#print(I)
# len(I)

# Can merge all elements of a tuple together as one string to obtain tuples of same dimension
Y=[]
for i in I:
    i = str(i)
    Y.append(''.join(i))
    
#print(Y)

#for y in Y:
#    print(len(y))

# TO DO : CAN TAKE A RANDOM SAMPLE OUT OF THIS SET TO BUILD A SMALLER NETWORK + graphic representation 


In [21]:
# Set of aircraft models
A = ['B738','B773','A321','A346']

In [25]:
# Assigning a random capacity to each aircraft model 
np.random.seed(12345)
K = {}
for a in A:
    K[a]=np.random.randint(200,500)

{'B738': 485, 'B773': 329, 'A321': 234, 'A346': 229}

In [110]:
# Assigning random demand to each itinerary
# -> from normal distribution ----> THEN WITH MULTINOMIAL LOGIT
np.random.seed(12345)
D={}
for i in Y:
    D[i]=np.random.normal(100, 10)/100


In [111]:
# Assigning random values for pollution to each aircraft type when on a specific itinerary 
# NOTE: WOULD MAKE MORE SENSE TO ASSIGN HIGHER VALUES TO LONGER ITINERARIES RATHER THAN COMPLETELY RANDOM
P = {}
for i in Y:
    for a in A:
        P[i, a]=np.random.randint(50, 1000)

In [112]:
# Creating a dummy for whether a flight-leg can be executed or not with a specific aircraft model:
# (0.5 probability of being yes/1)
R = {}
for i in Y:
    for a in A:
        if np.random.randn()<=0.5:
            R[i, a]=1
        else:
            R[i, a]=0

In [113]:
# Assigning random values for max utilization of each aircraft type
W = {}
for a in A:
    W[a] = np.random.randint(10, 50)

In [116]:
# NOT WORKING !!!! SET I CONTAINS ELEMENTS OF DIFFERENT DIMENSIONS, HOW DO YOU SOLVE THIS?
# TEMPORARY SOLUTION: using Y instead of I (itineraries as strings rather than integers)

# SETTING UP THE CONCRETE MODEL 
# initializing parmaters and sets
m = ConcreteModel()
m.I = Set(initialize=Y) # with 'dimen='' can only specify one dimension 
m.A = Set(initialize=A)
m.K=Param(m.A, initialize=K)
m.W = Param(m.A, initialize = W)
m.D=Param(m.I, initialize=D)
m.P=Param(m.I, m.A, initialize=P)
m.R=Param(m.I, m.A, initialize=R)

# setting up decision variables
m.f=Var(m.I, m.A, domain=NonNegativeIntegers) # f = frequency of flights operated by aircraft type A on itinerary I 
m.z=Var(m.A, domain=NonNegativeIntegers) # z= number of aircrafts type A

# setting up model objective and constraints
# OBJECTIVE
def minimize_pollution(m):
    return sum(m.f[i,a]*m.P[i,a]
              for i in m.I for a in m.A)
m.objective=Objective(rule=minimize_pollution, sense=minimize)

# CONSTRAINTS 
def utilization_rule(m, a):
    return sum(m.f[i,a]
              for i in m.I)<=m.W[a]*m.z[a]
m.utilization= Constraint(m.A, rule=utilization_rule)

def demand_rule(m, i):
    return sum(m.f[i, a]*m.K[a]
              for a in m.A)>=m.D[i]
m.demand_rule = Constraint(m.I, rule = demand_rule)

def operational_rule(model, i):
    return sum(m.f[i,a] 
              for I in m.I)<=m.R[i,a]*m.f[i,a]
m.operational_rule = Constraint(m.I, rule=operational_rule)


In [118]:
# calling the solver -> using cbc

#%%time # what do we need this for?
opt = SolverFactory('cbc', executable=r'/usr/local/bin/cbc')
opt.options['mipgap']=0.20 
results = opt.solve(m, tee=False) # tee=True to see the output of the model

In [127]:
# PRINTING SOME RESULTS
# 1. number of aircrafts for each model type
for a in m.A:
    print('Aircraft type: '+str(a))
    print('Number of aircrafts: '+str(m.z[a].value))

# 2. frequancy of flights on itinerary i operated with aircraft type a
for i in m.I:
    for a in m.A:
        print('Flight leg: %s; \naircraft type: %s; \nnumber of flights operated: %s\n' 
             % (i, a, m.f[i,a].value))

Aircraft type: B738
Number of aircrafts: 34.0
Aircraft type: B773
Number of aircrafts: 29.0
Aircraft type: A321
Number of aircrafts: 28.0
Aircraft type: A346
Number of aircrafts: 0.0
Flight leg: (1, 2); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (1, 2); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (1, 2); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (1, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (1, 3); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (1, 3); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (1, 3); 
aircraft type: A321; 
number of flights operated: 1.0

Flight leg: (1, 3); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (1, 4); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (1, 4); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (1, 4); 
aircraft type: A321; 
number of flig

Flight leg: (6, 5, 9); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (6, 7, 1); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (6, 7, 1); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (6, 7, 1); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (6, 7, 1); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (6, 7, 2); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (6, 7, 2); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (6, 7, 2); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (6, 7, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (6, 7, 3); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (6, 7, 3); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (6, 7, 3); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (6, 7, 3); 
aircraft type: A346; 
number

Flight leg: (8, 1, 7, 4); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (9, 1, 7, 4); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (9, 1, 7, 4); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (9, 1, 7, 4); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (9, 1, 7, 4); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (2, 1, 7, 5); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (2, 1, 7, 5); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (2, 1, 7, 5); 
aircraft type: A321; 
number of flights operated: 1.0

Flight leg: (2, 1, 7, 5); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (3, 1, 7, 5); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (3, 1, 7, 5); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (3, 1, 7, 5); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (3, 

Flight leg: (5, 2, 6, 4); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (7, 2, 6, 4); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (7, 2, 6, 4); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (7, 2, 6, 4); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (7, 2, 6, 4); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (8, 2, 6, 4); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (8, 2, 6, 4); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (8, 2, 6, 4); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (8, 2, 6, 4); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (9, 2, 6, 4); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (9, 2, 6, 4); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (9, 2, 6, 4); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (9, 

Flight leg: (1, 3, 9, 2); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (1, 3, 9, 2); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (1, 3, 9, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (4, 3, 9, 2); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (4, 3, 9, 2); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (4, 3, 9, 2); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (4, 3, 9, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (5, 3, 9, 2); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (5, 3, 9, 2); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (5, 3, 9, 2); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (5, 3, 9, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (6, 3, 9, 2); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (6, 


Flight leg: (5, 4, 9, 3); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (5, 4, 9, 3); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (5, 4, 9, 3); 
aircraft type: A321; 
number of flights operated: 1.0

Flight leg: (5, 4, 9, 3); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (6, 4, 9, 3); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (6, 4, 9, 3); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (6, 4, 9, 3); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (6, 4, 9, 3); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (7, 4, 9, 3); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (7, 4, 9, 3); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (7, 4, 9, 3); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (7, 4, 9, 3); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (8,

Flight leg: (8, 6, 2, 1); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (8, 6, 2, 1); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (8, 6, 2, 1); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (9, 6, 2, 1); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (9, 6, 2, 1); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (9, 6, 2, 1); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (9, 6, 2, 1); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (1, 6, 2, 3); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (1, 6, 2, 3); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (1, 6, 2, 3); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (1, 6, 2, 3); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (4, 6, 2, 3); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (4, 


Flight leg: (4, 7, 3, 1); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (4, 7, 3, 1); 
aircraft type: A321; 
number of flights operated: 1.0

Flight leg: (4, 7, 3, 1); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (5, 7, 3, 1); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (5, 7, 3, 1); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (5, 7, 3, 1); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (5, 7, 3, 1); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (6, 7, 3, 1); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (6, 7, 3, 1); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (6, 7, 3, 1); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (6, 7, 3, 1); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (8, 7, 3, 1); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (8,

Flight leg: (4, 8, 3, 2); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (4, 8, 3, 2); 
aircraft type: A321; 
number of flights operated: 1.0

Flight leg: (4, 8, 3, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (5, 8, 3, 2); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (5, 8, 3, 2); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (5, 8, 3, 2); 
aircraft type: A321; 
number of flights operated: 1.0

Flight leg: (5, 8, 3, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (6, 8, 3, 2); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (6, 8, 3, 2); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (6, 8, 3, 2); 
aircraft type: A321; 
number of flights operated: 1.0

Flight leg: (6, 8, 3, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (7, 8, 3, 2); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (7, 

Flight leg: (8, 9, 4, 1); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (8, 9, 4, 1); 
aircraft type: A321; 
number of flights operated: 1.0

Flight leg: (8, 9, 4, 1); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (1, 9, 4, 2); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (1, 9, 4, 2); 
aircraft type: B773; 
number of flights operated: 0.0

Flight leg: (1, 9, 4, 2); 
aircraft type: A321; 
number of flights operated: 1.0

Flight leg: (1, 9, 4, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (3, 9, 4, 2); 
aircraft type: B738; 
number of flights operated: 0.0

Flight leg: (3, 9, 4, 2); 
aircraft type: B773; 
number of flights operated: 1.0

Flight leg: (3, 9, 4, 2); 
aircraft type: A321; 
number of flights operated: 0.0

Flight leg: (3, 9, 4, 2); 
aircraft type: A346; 
number of flights operated: 0.0

Flight leg: (5, 9, 4, 2); 
aircraft type: B738; 
number of flights operated: 1.0

Flight leg: (5, 